In [43]:
import numpy as np
import cupy as cp
from numba import cuda


In [44]:
cuda.detect()

Found 1 CUDA devices
id 0            b'Tesla K80'                              [SUPPORTED]
                      compute capability: 3.7
                           pci device id: 4
                              pci bus id: 0
Summary:
	1/1 devices are supported


True

In [45]:
#create a CPU numpy array (we suppose that we have these data only on CPU and we can not generate them directly on GPU)
h_A = np.random.randint(0, 255, size=(1000, 1000))
h_B = np.random.randint(0, 255, size=(1000, 1000))

#we generate the result matrix directly on GPU
d_C = cp.zeros((1000, 1000) , dtype=np.float64)

#size of the cpu_array
size_array_in_memory = (h_A.nbytes / 1e6) + (h_B.nbytes / 1e6)
print(f"the size in memory is:{size_array_in_memory} Mega byte\n")

#Sending the array to device
d_A = cuda.to_device(h_A)
d_B = cuda.to_device(h_B)

the size in memory is:16.0 Mega byte



In [46]:
@cuda.jit
def matmul(A, B, C):
    """
      Perform square matrix multiplication of C = A * B
      this kernel is executed on each element of our matrix
    """
    i, j = cuda.grid(2)  
    if i < C.shape[0] and j < C.shape[1]:   # grid can extend beyond C
        tmp = 0.      
        for k in range(A.shape[1]):
            tmp += A[i, k] * B[k, j]        # multiply elements in row i of A and column j of B and add to temp
        C[i, j] = tmp

In [47]:
#Compute thread and grid to execute
threadsperblock = (16, 16)  # each block will contain 16x16 threads, typically 128 - 512 threads/block
blockspergrid_x = int(np.ceil(d_C.shape[0] / threadsperblock[0]))
blockspergrid_y = int(np.ceil(d_C.shape[1] / threadsperblock[1]))
blockspergrid = (blockspergrid_x, blockspergrid_y)  # we calculate the gridsize (number of blocks) from array
print(blockspergrid)
print(f"The kernel will be executed up to element {threadsperblock[0]*blockspergrid_x}")

(63, 63)
The kernel will be executed up to element 1008


In [48]:
# execution of the kernel
matmul[blockspergrid, threadsperblock](d_A, d_B, d_C)

#print the data A with cupy
print(cp.asarray(d_C))
print("-------\n")

[[15407195. 16137516. 15955002. ... 16327436. 15955913. 16199674.]
 [15855426. 16203021. 16429476. ... 16859972. 16359792. 17032604.]
 [16281369. 15744481. 16302085. ... 16641918. 16251003. 16837721.]
 ...
 [16033470. 16512866. 16728134. ... 16900391. 16746713. 16910387.]
 [15894458. 15649875. 16158911. ... 16114632. 15648543. 16198288.]
 [15957473. 15915692. 16382346. ... 16549592. 16131364. 17014892.]]
-------



In [53]:
# faster multiplication can be obtained by making use of shared memory between threads in the same block
# this requires more thinking about non-obvious implementation!

from numba import float32, int32, float64

# Controls threads per block and shared memory usage.
# The computation will be done on blocks of TPBxTPB elements.
TPB = 16

@cuda.jit
def fast_matmul(A, B, C):
    # Define an array in the shared memory
    # The size and type of the arrays must be known at compile time
    sA = cuda.shared.array(shape=(TPB, TPB), dtype=float32)
    sB = cuda.shared.array(shape=(TPB, TPB), dtype=float32)

    x, y = cuda.grid(2)

    tx = cuda.threadIdx.x
    ty = cuda.threadIdx.y
    bpg = cuda.gridDim.x    # blocks per grid

    if x >= C.shape[0] and y >= C.shape[1]:
        # Quit if (x, y) is outside of valid C boundary
        return

    # Each thread computes one element in the result matrix.
    # The dot product is chunked into dot products of TPB-long vectors.
    tmp = 0.
    for i in range(bpg):
        # Preload data into shared memory
        sA[tx, ty] = A[x, ty + i * TPB]
        sB[tx, ty] = B[tx + i * TPB, y]

        # Wait until all threads finish preloading
        cuda.syncthreads()

        # Computes partial product on the shared memory
        for j in range(TPB):
            tmp += sA[tx, j] * sB[j, ty]

        # Wait until all threads finish computing
        cuda.syncthreads()

    C[x, y] = tmp

In [55]:
# execution of the kernel
fast_matmul[blockspergrid, threadsperblock](d_A, d_B, d_C)

#print the data A with cupy
print(cp.asarray(d_C))
print("-------\n")

[[15407195. 16137516. 15955002. ... 16327436. 15955913. 16199674.]
 [15682464. 15757583. 16270867. ... 16859972. 16359792. 17032604.]
 [15653544. 16160277. 16286697. ... 16641918. 16251003. 16837721.]
 ...
 [ 7444758.  7823575.  8036345. ... 16900391. 16746713. 16910387.]
 [ 8054330.  8626057.  8710958. ... 16114632. 15648543. 16198288.]
 [ 7744923.  8119763.  8262746. ... 16549592. 16131364. 17014892.]]
-------



In [56]:
#get back the matrix to host
h_C = d_C.get()
print(h_C)

[[15407195. 16137516. 15955002. ... 16327436. 15955913. 16199674.]
 [15682464. 15757583. 16270867. ... 16859972. 16359792. 17032604.]
 [15653544. 16160277. 16286697. ... 16641918. 16251003. 16837721.]
 ...
 [ 7444758.  7823575.  8036345. ... 16900391. 16746713. 16910387.]
 [ 8054330.  8626057.  8710958. ... 16114632. 15648543. 16198288.]
 [ 7744923.  8119763.  8262746. ... 16549592. 16131364. 17014892.]]
